# 主程序 0：3-10 顶点图最大割数求解

In [1]:
import numpy as np
import networkx as nx
import mindspore as ms
from mindquantum import *
from mindspore import nn, ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
np.random.seed(1)

# 构造图结构

for node_num in [3, 4, 5, 6, 7, 8, 9, 10]: # 顶点数

    edges = []
    # 添加环边
    for node in range(node_num-1):
        edges.append([node, node+1])
    edges.append([node_num-1, 0])

     # 添加内边
    if node_num % 2 == 0: # 如果顶点数为偶数，就采用 3-regular 结构
        for node in range(int(node_num/2)):
            edges.append([node, int(node+node_num/2)])
    else:                               # 如果顶点为奇数，只留一个顶点有两条边，其他为三条边
        for node in range(int((node_num-1)/2)):
            edges.append([node, int(node+(node_num-1)/2)])

    print('=================='*3)
    print(node_num, '顶点图，', '边包括：', edges)

    # print('\n图形为：')
    # nx.draw(g, with_labels=True, font_weight='bold') # 画出图

    ## 经典穷举法
    g = nx.Graph()
    for i in range(len(edges)):
        nx.add_path(g, edges[i])

    max_cut = 0 # 通过比较，得到最大割数
    for i in g.nodes:
        max_cut = max(max_cut, nx.cut_size(g, [i]))       # 一组 1 个顶点、另一组 node_num - 1 个顶点
        for j in range(i):
            max_cut = max(max_cut, nx.cut_size(g, [i, j]))  # 一组 2 个顶点、另一组 node_num - 2 个顶点
            if node_num > 5:
                for k in range(j):
                    max_cut = max(max_cut, nx.cut_size(g, [i, j, k]))  # 一组 3 个顶点、另一组 node_num - 3 个顶点
                    if node_num > 7:
                        for m in range(k):
                            max_cut = max(max_cut, nx.cut_size(g, [i, j, k, m]))  # 一组 4 个顶点、另一组 node_num - 4 个顶点
                            if node_num > 9:
                                for n in range(m):
                                    max_cut = max(max_cut, nx.cut_size(g, [i, j, k, m, n]))  # 一组 5 个顶点、另一组 node_num - 5 个顶点

    print('经典穷举法的最大割数为：', max_cut)

    ## 量子 MBE 法
    # 量子网络的基本层结构
    def QLayer(qubit_num=4, prefix='0'):
        circ_ = Circuit()
        for qubit in range(qubit_num):
            circ_ += RY(f'0_{qubit}').on(qubit)
        for qubit in range(0, qubit_num-1, 2):
            circ_ += Z.on(qubit+1, qubit)
        for qubit in range(qubit_num):
            circ_ += RY(f'1_{qubit}').on(qubit)
        for qubit in range(1, qubit_num-1, 2):
            circ_ += Z.on(qubit+1, qubit)
        circ_ += Z.on(0, qubit_num-1)
        circ_ = add_prefix(circ_, prefix)
        return circ_

    qubit_num = int((node_num + 1) / 2) # 比特数为顶点数的一半
    layer_num = 2 * qubit_num # 层数为比特数的两倍

    ansatz = Circuit()
    for layer in range(layer_num): # 拟设结构
        ansatz += QLayer(qubit_num=qubit_num, prefix=f'{layer}')

    sim = Simulator('mqvector', ansatz.n_qubits)
    if node_num % 2 == 0:
        ham =  [Hamiltonian(QubitOperator(f'Z{i}')) for i in range(qubit_num)] + \
                    [Hamiltonian(QubitOperator(f'X{i}')) for i in range(qubit_num)]
    else:
        ham =  [Hamiltonian(QubitOperator(f'Z{i}')) for i in range(qubit_num-1)] + \
                    [Hamiltonian(QubitOperator(f'X{i}')) for i in range(qubit_num-1)] + \
                    [Hamiltonian(QubitOperator(f'Z{qubit_num-1}'))]
    grad_ops = sim.get_expectation_with_grad(ham, ansatz)

    class MyLoss(nn.LossBase): # 构建损失函数
        def __init__(self, reduction='mean'):
            super(MyLoss, self).__init__(reduction)
            self.tanh = ops.Tanh()

        def construct(self, logits):
            x = self.tanh(logits)
            out = 0
            for edge in edges:
                out += x[edge[0]] * x[edge[1]]
            return self.get_loss(out)

    class MyWithLossCell(nn.Cell): # 量子网络与损失函数结合
        def __init__(self, backbone, loss_fn):
            super(MyWithLossCell, self).__init__(auto_prefix=False)
            self._backbone = backbone
            self._loss_fn = loss_fn

        def construct(self):
            out = self._backbone()
            return self._loss_fn(out)

    QuantumNet = MQAnsatzOnlyLayer(grad_ops)
    loss = MyLoss()
    net_with_criterion = MyWithLossCell(QuantumNet, loss)
    opti = nn.Adam(QuantumNet.trainable_params(), learning_rate=0.05)
    net = nn.TrainOneStepCell(net_with_criterion, opti)

    # 训练 200 次
    for i in range(200):
        res = net()

    round = ops.Round()
    out = QuantumNet()
    result = 0
    for edge in edges:
        result += (1 - round(out[edge[0]]) * round(out[edge[1]])) / 2

    print('量子 MBE 法得到的最大割数为：',  int(result.asnumpy() + 0.5)) # 最后 +0.5 是为了完善 int 函数的作用

[WARNING] ME(931614:139863881864512,MainProcess):2023-10-17-19:57:11.462.888 [mindspore/run_check/_check_version.py:102] MindSpore version 2.1.0 and cuda version 11.8.89 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.


3 顶点图， 边包括： [[0, 1], [1, 2], [2, 0], [0, 1]]
经典穷举法的最大割数为： 2
量子 MBE 法得到的最大割数为： 3
4 顶点图， 边包括： [[0, 1], [1, 2], [2, 3], [3, 0], [0, 2], [1, 3]]
经典穷举法的最大割数为： 4
量子 MBE 法得到的最大割数为： 4
5 顶点图， 边包括： [[0, 1], [1, 2], [2, 3], [3, 4], [4, 0], [0, 2], [1, 3]]
经典穷举法的最大割数为： 6
量子 MBE 法得到的最大割数为： 6
6 顶点图， 边包括： [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 0], [0, 3], [1, 4], [2, 5]]
经典穷举法的最大割数为： 9
量子 MBE 法得到的最大割数为： 9
7 顶点图， 边包括： [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 0], [0, 3], [1, 4], [2, 5]]
经典穷举法的最大割数为： 9
量子 MBE 法得到的最大割数为： 9
8 顶点图， 边包括： [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 0], [0, 4], [1, 5], [2, 6], [3, 7]]
经典穷举法的最大割数为： 10
量子 MBE 法得到的最大割数为： 10
9 顶点图， 边包括： [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 0], [0, 4], [1, 5], [2, 6], [3, 7]]
经典穷举法的最大割数为： 12
量子 MBE 法得到的最大割数为： 12
10 顶点图， 边包括： [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 0], [0, 5], [1, 6], [2, 7], [3, 8], [4, 9]]
经典穷举法的最大割数为： 15
量子 MBE 法得到的最大割数为： 15


以上结果显示，MBE 法得到的最大割数与经典法一致 。

# 主程序 1：用同一个量子网络，计算不同图的最大割问题

In [2]:
import numpy as np
import networkx as nx
import mindspore as ms
from mindquantum import *
from mindspore import nn, ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
np.random.seed(1)

## 构造图结构

node_num = 4 # 顶点数

# 两个不同的图结构
edges_0 = [[0, 1], [1, 2], [2, 3], [0, 2], [1, 3]]
edges_1 = [[0, 1], [1, 2], [0, 3], [1, 3]]

print('图的顶点数为：', node_num)
print( '两种图结构分别为：')
print(edges_0)
print(edges_1)

## 经典穷举法
g_0 = nx.Graph()
for i in range(len(edges_0)):
    nx.add_path(g_0, edges_0[i])

g_1 = nx.Graph()
for i in range(len(edges_1)):
    nx.add_path(g_1, edges_1[i])

max_cut_0= 0 # 通过比较，得到最大割数
for i in g_0.nodes:
    max_cut_0 = max(max_cut_0, nx.cut_size(g_0, [i]))       # 一组 1 个顶点、另一组 node_num - 1 个顶点
    for j in range(i):
        max_cut_0 = max(max_cut_0, nx.cut_size(g_0, [i, j]))  # 一组 2 个顶点、另一组 node_num - 2 个顶点

max_cut_1= 0 # 通过比较，得到最大割数
for i in g_1.nodes:
    max_cut_1 = max(max_cut_1, nx.cut_size(g_1, [i]))       # 一组 1 个顶点、另一组 node_num - 1 个顶点
    for j in range(i):
        max_cut_1 = max(max_cut_1, nx.cut_size(g_1, [i, j]))  # 一组 2 个顶点、另一组 node_num - 2 个顶点

print('\n经典穷举法对第一种图结构的最大割数为：', max_cut_0)
print('经典穷举法对第二种图结构的最大割数为：', max_cut_1)

## 量子 MBE 法
# 量子网络的基本层结构
def QLayer(qubit_num=4, prefix='0'):
    circ_ = Circuit()
    for qubit in range(qubit_num):
        circ_ += RY(f'0_{qubit}').on(qubit)
    for qubit in range(0, qubit_num-1, 2):
        circ_ += Z.on(qubit+1, qubit)
    for qubit in range(qubit_num):
        circ_ += RY(f'1_{qubit}').on(qubit)
    for qubit in range(1, qubit_num-1, 2):
        circ_ += Z.on(qubit+1, qubit)
    circ_ += Z.on(0, qubit_num-1)
    circ_ = add_prefix(circ_, prefix)
    return circ_

qubit_num = int(node_num / 2) # 比特数为顶点数的一半
layer_num = 2 * qubit_num # 层数为比特数的两倍

ansatz = Circuit()
for layer in range(layer_num): # 拟设结构
    ansatz += QLayer(qubit_num=qubit_num, prefix=f'{layer}')

sim = Simulator('mqvector', ansatz.n_qubits)
ham =  [Hamiltonian(QubitOperator(f'Z{i}')) for i in range(qubit_num)] + \
            [Hamiltonian(QubitOperator(f'X{i}')) for i in range(qubit_num)]
grad_ops = sim.get_expectation_with_grad(ham, ansatz)

class MyLoss_0(nn.LossBase): # 针对第一种图结构构建损失函数
    def __init__(self, reduction='mean'):
        super(MyLoss_0, self).__init__(reduction)
        self.tanh = ops.Tanh()

    def construct(self, logits):
        x = self.tanh(logits)
        out = 0
        for edge in edges_0:
            out += x[edge[0]] * x[edge[1]]
        return self.get_loss(out)

class MyLoss_1(nn.LossBase): # 针对第二种图结构构建损失函数
    def __init__(self, reduction='mean'):
        super(MyLoss_1, self).__init__(reduction)
        self.tanh = ops.Tanh()

    def construct(self, logits):
        x = self.tanh(logits)
        out = 0
        for edge in edges_1:
            out += x[edge[0]] * x[edge[1]]
        return self.get_loss(out)

class MyWithLossCell(nn.Cell): # 量子网络与损失函数结合
    def __init__(self, backbone, loss_fn):
        super(MyWithLossCell, self).__init__(auto_prefix=False)
        self._backbone = backbone
        self._loss_fn = loss_fn

    def construct(self):
        out = self._backbone()
        return self._loss_fn(out)

QuantumNet_0 = MQAnsatzOnlyLayer(grad_ops)
QuantumNet_1 = MQAnsatzOnlyLayer(grad_ops)
loss_0 = MyLoss_0()
loss_1 = MyLoss_1()
net_with_criterion_0 = MyWithLossCell(QuantumNet_0, loss_0)
net_with_criterion_1 = MyWithLossCell(QuantumNet_1, loss_1)
opti_0 = nn.Adam(QuantumNet_0.trainable_params(), learning_rate=0.05)
opti_1 = nn.Adam(QuantumNet_1.trainable_params(), learning_rate=0.05)
net_0 = nn.TrainOneStepCell(net_with_criterion_0, opti_0)
net_1 = nn.TrainOneStepCell(net_with_criterion_1, opti_1)

# 同时训练 200 次
for i in range(200):
    net_0()
    net_1()

round = ops.Round()
out_0 = QuantumNet_0()
out_1 = QuantumNet_1()
result_0 = 0
result_1 = 0

for edge in edges_0:
    result_0 += (1 - round(out_0[edge[0]]) * round(out_0[edge[1]])) / 2

for edge in edges_1:
    result_1 += (1 - round(out_1[edge[0]]) * round(out_1[edge[1]])) / 2

print('\n量子 MBE 法对第一种图结构得到的最大割数为：',  int(result_0.asnumpy() + 0.5)) # 最后 +0.5 是为了完善 int 函数的作用
print('量子 MBE 法对第二种图结构得到的最大割数为：',  int(result_1.asnumpy() + 0.5)) # 最后 +0.5 是为了完善 int 函数的作用


图的顶点数为： 4
两种图结构分别为：
[[0, 1], [1, 2], [2, 3], [0, 2], [1, 3]]
[[0, 1], [1, 2], [0, 3], [1, 3]]

经典穷举法对第一种图结构的最大割数为： 4
经典穷举法对第二种图结构的最大割数为： 3

量子 MBE 法对第一种图结构得到的最大割数为： 4
量子 MBE 法对第二种图结构得到的最大割数为： 3


以上结果显示，对同一个量子网络，通过根据所求图结构构造不同的目标函数，可以同时进行求解。也即，采用同一个量子网络，可以求解不同图结构的最大割问题。